In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 20
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 20
N_FEATURES = 10

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [22]:
verbose, epochs, batch_size = 0, 120, 64

In [23]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [24]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [25]:
import os
path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/'

In [26]:
for i in range(1,28):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 20, 10
    n_depth=1
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(9,1), padding='same', kernel_initializer="he_normal",strides=(3,1), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(MaxPooling2D(pool_size=(4,1),strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Conv2D(filters=64, kernel_size=(5,1),padding="same",kernel_initializer="he_normal",strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2093))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='tanh'))
    model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=2.17e-3, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
#     print(test_acc)
    test_mean=statistics.mean(test_acc)
    print(test_mean)

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 182000

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 78000

x_train shape:  (9099, 20, 10)
9099 training samples
y_train shape:  (9099,)
num_time_periods 20
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (20, 10)
input_shape: (20, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (9099, 52)
x

Epoch 15/120
133/143 [==========================>...] - ETA: 0s - loss: 0.9655 - accuracy: 0.6955
Epoch 00015: val_accuracy did not improve from 0.62324
143/143 [==============================] - 1s 8ms/step - loss: 0.9633 - accuracy: 0.6948 - val_loss: 1.3615 - val_accuracy: 0.5932 - lr: 0.0010
Epoch 16/120
134/143 [===========================>..] - ETA: 0s - loss: 0.9373 - accuracy: 0.7017
Epoch 00016: val_accuracy did not improve from 0.62324
143/143 [==============================] - 1s 8ms/step - loss: 0.9418 - accuracy: 0.7000 - val_loss: 1.3205 - val_accuracy: 0.6194 - lr: 0.0010
Epoch 17/120
133/143 [==========================>...] - ETA: 0s - loss: 0.9089 - accuracy: 0.7165
Epoch 00017: val_accuracy improved from 0.62324 to 0.63555, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
143/143 [==============================] - 1s 8ms/step - loss: 0.9114 - accuracy: 0.7152 - val_loss: 1.2602 - val_accuracy: 0.6355 - lr: 0.0

Epoch 40/120
140/143 [============================>.] - ETA: 0s - loss: 0.4447 - accuracy: 0.8594
Epoch 00040: val_accuracy improved from 0.67889 to 0.68582, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
143/143 [==============================] - 1s 8ms/step - loss: 0.4442 - accuracy: 0.8594 - val_loss: 1.2027 - val_accuracy: 0.6858 - lr: 1.0000e-04
Epoch 41/120
137/143 [===========================>..] - ETA: 0s - loss: 0.4189 - accuracy: 0.8704
Epoch 00041: val_accuracy improved from 0.68582 to 0.69249, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
143/143 [==============================] - 1s 9ms/step - loss: 0.4174 - accuracy: 0.8711 - val_loss: 1.1964 - val_accuracy: 0.6925 - lr: 1.0000e-04
Epoch 42/120
141/143 [============================>.] - ETA: 0s - loss: 0.4139 - accuracy: 0.8707
Epoch 00042: val_accuracy improved from 0.69249 to 0.69300, saving model t

Epoch 65/120
138/143 [===========================>..] - ETA: 0s - loss: 0.3836 - accuracy: 0.8783
Epoch 00065: val_accuracy did not improve from 0.69633
143/143 [==============================] - 1s 7ms/step - loss: 0.3835 - accuracy: 0.8783 - val_loss: 1.2046 - val_accuracy: 0.6874 - lr: 1.0000e-04
Epoch 66/120
139/143 [============================>.] - ETA: 0s - loss: 0.3800 - accuracy: 0.8804
Epoch 00066: val_accuracy did not improve from 0.69633
143/143 [==============================] - 1s 7ms/step - loss: 0.3781 - accuracy: 0.8811 - val_loss: 1.2138 - val_accuracy: 0.6876 - lr: 1.0000e-04
Epoch 67/120
135/143 [===========================>..] - ETA: 0s - loss: 0.3797 - accuracy: 0.8800
Epoch 00067: val_accuracy did not improve from 0.69633
143/143 [==============================] - 1s 7ms/step - loss: 0.3777 - accuracy: 0.8803 - val_loss: 1.2180 - val_accuracy: 0.6935 - lr: 1.0000e-04
Epoch 68/120
137/143 [===========================>..] - ETA: 0s - loss: 0.3771 - accuracy: 0.8814

143/143 [==============================] - ETA: 0s - loss: 0.3517 - accuracy: 0.8886
Epoch 00119: val_accuracy did not improve from 0.69633
143/143 [==============================] - 1s 7ms/step - loss: 0.3517 - accuracy: 0.8886 - val_loss: 1.2236 - val_accuracy: 0.6945 - lr: 1.0000e-05
Epoch 120/120
140/143 [============================>.] - ETA: 0s - loss: 0.3459 - accuracy: 0.8958
Epoch 00120: val_accuracy did not improve from 0.69633
143/143 [==============================] - 1s 8ms/step - loss: 0.3457 - accuracy: 0.8957 - val_loss: 1.2236 - val_accuracy: 0.6953 - lr: 1.0000e-06
epoch_number 55
train accuracy and validation accuracy 0.8810858130455017 0.6963323950767517
61/61 [==============================] - 0s 4ms/step - loss: 1.1908 - accuracy: 0.6963
test_accuracy 0.6963323950767517
[0.6963323950767517]
test_mean for %d subjects: 1
0.6963323950767517
/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S2_tr.csv
Number of columns in the dataframe: 11

...

143/143 [==============================] - ETA: 0s - loss: 0.6220 - accuracy: 0.8171
Epoch 00074: val_accuracy did not improve from 0.57399
143/143 [==============================] - 1s 9ms/step - loss: 0.6220 - accuracy: 0.8171 - val_loss: 1.7574 - val_accuracy: 0.5727 - lr: 1.0000e-04
Epoch 75/120
139/143 [============================>.] - ETA: 0s - loss: 0.6209 - accuracy: 0.8192
Epoch 00075: val_accuracy did not improve from 0.57399
143/143 [==============================] - 1s 8ms/step - loss: 0.6222 - accuracy: 0.8191 - val_loss: 1.7626 - val_accuracy: 0.5673 - lr: 1.0000e-04
Epoch 76/120
140/143 [============================>.] - ETA: 0s - loss: 0.6148 - accuracy: 0.8212
Epoch 00076: val_accuracy did not improve from 0.57399
143/143 [==============================] - 1s 7ms/step - loss: 0.6155 - accuracy: 0.8210 - val_loss: 1.7596 - val_accuracy: 0.5681 - lr: 1.0000e-04
Epoch 77/120
136/143 [===========================>..] - ETA: 0s - loss: 0.6195 - accuracy: 0.8178
Epoch 00077:

141/143 [============================>.] - ETA: 0s - loss: 2.8910 - accuracy: 0.2547
Epoch 00001: val_accuracy improved from -inf to 0.37009, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
143/143 [==============================] - 1s 8ms/step - loss: 2.8873 - accuracy: 0.2556 - val_loss: 2.3218 - val_accuracy: 0.3701 - lr: 0.0010
Epoch 2/120
136/143 [===========================>..] - ETA: 0s - loss: 2.0971 - accuracy: 0.4099
Epoch 00002: val_accuracy improved from 0.37009 to 0.43268, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
143/143 [==============================] - 1s 7ms/step - loss: 2.0915 - accuracy: 0.4102 - val_loss: 2.0394 - val_accuracy: 0.4327 - lr: 0.0010
Epoch 3/120
138/143 [===========================>..] - ETA: 0s - loss: 1.8349 - accuracy: 0.4789
Epoch 00003: val_accuracy improved from 0.43268 to 0.48654, saving model to /media/naveen/nav/mat_co

Epoch 24/120
141/143 [============================>.] - ETA: 0s - loss: 0.8562 - accuracy: 0.7339
Epoch 00024: val_accuracy improved from 0.59938 to 0.60195, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
143/143 [==============================] - 1s 8ms/step - loss: 0.8553 - accuracy: 0.7340 - val_loss: 1.4989 - val_accuracy: 0.6019 - lr: 0.0010
Epoch 25/120
143/143 [==============================] - ETA: 0s - loss: 0.8338 - accuracy: 0.7422
Epoch 00025: val_accuracy did not improve from 0.60195
143/143 [==============================] - 1s 8ms/step - loss: 0.8338 - accuracy: 0.7422 - val_loss: 1.4896 - val_accuracy: 0.5971 - lr: 0.0010
Epoch 26/120
138/143 [===========================>..] - ETA: 0s - loss: 0.8050 - accuracy: 0.7512
Epoch 00026: val_accuracy did not improve from 0.60195
143/143 [==============================] - 1s 7ms/step - loss: 0.8033 - accuracy: 0.7509 - val_loss: 1.5058 - val_accuracy: 0.5976 - lr: 0.0

Epoch 48/120
136/143 [===========================>..] - ETA: 0s - loss: 0.4667 - accuracy: 0.8619
Epoch 00048: val_accuracy did not improve from 0.63811
143/143 [==============================] - 1s 6ms/step - loss: 0.4678 - accuracy: 0.8619 - val_loss: 1.4501 - val_accuracy: 0.6335 - lr: 1.0000e-04
Epoch 49/120
136/143 [===========================>..] - ETA: 0s - loss: 0.4756 - accuracy: 0.8588
Epoch 00049: val_accuracy did not improve from 0.63811
143/143 [==============================] - 1s 7ms/step - loss: 0.4773 - accuracy: 0.8575 - val_loss: 1.4517 - val_accuracy: 0.6379 - lr: 1.0000e-04
Epoch 50/120
141/143 [============================>.] - ETA: 0s - loss: 0.4669 - accuracy: 0.8593
Epoch 00050: val_accuracy did not improve from 0.63811
143/143 [==============================] - 1s 7ms/step - loss: 0.4678 - accuracy: 0.8590 - val_loss: 1.4520 - val_accuracy: 0.6343 - lr: 1.0000e-04
Epoch 51/120
140/143 [============================>.] - ETA: 0s - loss: 0.4669 - accuracy: 0.8616

Epoch 75/120
139/143 [============================>.] - ETA: 0s - loss: 0.4352 - accuracy: 0.8721
Epoch 00075: val_accuracy did not improve from 0.63863
143/143 [==============================] - 1s 7ms/step - loss: 0.4355 - accuracy: 0.8714 - val_loss: 1.4776 - val_accuracy: 0.6345 - lr: 1.0000e-04
Epoch 76/120
143/143 [==============================] - ETA: 0s - loss: 0.4324 - accuracy: 0.8693
Epoch 00076: val_accuracy did not improve from 0.63863
143/143 [==============================] - 1s 7ms/step - loss: 0.4324 - accuracy: 0.8693 - val_loss: 1.4896 - val_accuracy: 0.6325 - lr: 1.0000e-04
Epoch 77/120
139/143 [============================>.] - ETA: 0s - loss: 0.4288 - accuracy: 0.8744
Epoch 00077: val_accuracy did not improve from 0.63863
143/143 [==============================] - 1s 7ms/step - loss: 0.4280 - accuracy: 0.8748 - val_loss: 1.4922 - val_accuracy: 0.6322 - lr: 1.0000e-04
Epoch 78/120
143/143 [==============================] - ETA: 0s - loss: 0.4259 - accuracy: 0.8731

In [27]:
statistics.mean(test_acc)

0.68068735687821